# Playing Frozen Lake with Q-learning

Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. 

At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend.

The surface is described using a grid like the following:

SFFF       
FHFH       
FFFH       
HFFG

S: starting point, safe  
F: frozen surface, safe  
H: hole, fall to your doom  
G: goal, where the frisbee is located

The episode ends when you reach the goal or fall in a hole.  
You receive a reward of 1 if you reach the goal, and 0 otherwise.

https://gymnasium.farama.org/environments/toy_text/frozen_lake/

## Notice

The team that has been maintaining Gym has moved all future development to Gymnasium. The original Gym repo isn't planned to receive any future updates. All future updates to the API will be in the Gymnasium repo. Please install Gymnasium with the command below and import gymnasium as gym. This notebook has been updated to use Gymnasium.  

pip install gymnasium  

Reference: https://github.com/openai/gym#important-notice

In [5]:
import numpy as np
import gymnasium as gym
import random
import time
from IPython.display import clear_output

In [6]:
env = gym.make("FrozenLake-v1", render_mode='ansi')

In [7]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [8]:
assert state_space_size == 16
assert action_space_size == 4

In [9]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [10]:
rewards_all_episodes = []

# Q-learning algorithm
for episode in range(num_episodes):
    state = env.reset()[0]
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):       
        
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()

        new_state, reward, done, truncated, info = env.step(action)

        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward        
        
        if done == True: 
            break
           
    # Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)    
    
    rewards_all_episodes.append(rewards_current_episode)

# Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000
print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    avg_reward = sum(r/1000)
    print(count, ": ", str(avg_reward))
    count += 1000    

# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)

********Average reward per thousand episodes********

1000 :  0.03200000000000002
2000 :  0.17100000000000012
3000 :  0.4100000000000003
4000 :  0.5670000000000004
5000 :  0.6030000000000004
6000 :  0.6780000000000005
7000 :  0.6690000000000005
8000 :  0.6620000000000005
9000 :  0.7050000000000005
10000 :  0.6690000000000005


********Q-table********

[[0.58210734 0.50210488 0.49332921 0.49808724]
 [0.34460977 0.40080961 0.3749955  0.51356631]
 [0.39571494 0.42215766 0.4057487  0.48314675]
 [0.35714404 0.32971016 0.32931546 0.45451062]
 [0.5932794  0.51994401 0.26928075 0.33332111]
 [0.         0.         0.         0.        ]
 [0.4543072  0.13825018 0.17492535 0.10037712]
 [0.         0.         0.         0.        ]
 [0.29640764 0.46206724 0.40284359 0.61743311]
 [0.30866506 0.69186541 0.37593483 0.42231526]
 [0.7247223  0.30048757 0.44144856 0.29540224]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.29073268 0.64462015 0.76774664 

In [11]:
assert avg_reward > 0.6

In [21]:
# Watch agent play Frozen Lake by playing the best action 
# from each state according to the Q-table.
# Run for more episodes to watch longer.

for episode in range(3):
    state = env.reset()[0]
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):        
        clear_output(wait=True)
        print(env.render())
        time.sleep(0.3)
        
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            print(env.render())
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
            clear_output(wait=True)
            break
            
        state = new_state
        
env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG

****You reached the goal!****
